In [24]:
## STEP ONE: EXTRACTION

In [20]:
import requests
import pandas as pd
import psycopg2
import json
import psycopg2
from dotenv import load_dotenv
import os

### STEP 2 : CARRY OUT THE EXTRACTION 

In [ ]:
#url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

#querystring = {"limit":"500"}

#headers = {
#	"x-rapidapi-key": "e64c48a3b5mshfe9cdd04e4c4a29p18e80bjsn55c3acbad527",
#	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
#}

#response = requests.get(url, headers=headers, params=querystring)

In [ ]:
# convert the data from jason format to table format
#print(response.json())

{'messages': 'The API is unreachable, please contact the API provider', 'info': 'Your Client (working) ---> Gateway (working) ---> API (not working)'}


In [ ]:
#data = response.json()
#file_name = 'OOreal_estate.json'

#with open(file_name, 'w') as file:
 #   json.dump(data, file, indent=4)

In [2]:
import json
import pandas as pd

with open('OOreal_estate.json') as f:
    data = json.load(f)

# If data is a dictionary of scalars:
real_estate_df = pd.DataFrame([data])


#### TO PUT THE JSON FILE IN A TABLER FORM 
#### FOLLOW THIS STEPS

In [3]:
real_estate_df =  pd.read_json('OOreal_estate.json')
real_estate_df.head()

,addressLine1,city,state,zipCode,formattedAddress,bedrooms,squareFootage,yearBuilt,features,county,...,propertyTaxes,lastSalePrice,lastSaleDate,owner,id,longitude,latitude,subdivision,zoning,addressLine2
0,511 John St,Bryan,OH,43506,"511 John St, Bryan, OH 43506",3.0,1404.0,1959.0,"{'architectureType': 'Conventional', 'cooling'...",Williams,...,{'2021': {'total': 1799}},105000.0,2000-10-13T00:00:00.000Z,"{'names': ['SUSAN JO KERR'], 'mailingAddress':...","511-John-St,-Bryan,-OH-43506",-84.570163,41.469595,NaN,NaN,NaN
1,1067 Marlin Dr,Rockledge,FL,32955,"1067 Marlin Dr, Rockledge, FL 32955",3.0,2258.0,1966.0,"{'cooling': True, 'coolingType': 'Central', 'e...",Brevard,...,"{'2021': {'total': 1830}, '2022': {'total': 21...",125000.0,2019-08-13T00:00:00.000Z,"{'names': ['GYULA L KISS', 'JULIE A KISS'], 'm...","1067-Marlin-Dr,-Rockledge,-FL-32955",-80.746982,28.342440,MARLIN SUBD,R2,NaN
2,1136 Sullivan Rd,Ogden,UT,84403,"1136 Sullivan Rd, Apt 15, Ogden, UT 84403",NaN,NaN,NaN,{},Weber,...,NaN,NaN,NaN,NaN,"1136-Sullivan-Rd,-Apt-15,-Ogden,-UT-84403",-111.950330,41.204860,NaN,NaN,Apt 15
3,101 Trinity Lakes Dr,Sun City Center,FL,33573,"101 Trinity Lakes Dr, Apt 361, Sun City Center...",NaN,NaN,NaN,{},Hillsborough,...,NaN,NaN,NaN,NaN,"101-Trinity-Lakes-Dr,-Apt-361,-Sun-City-Center...",-82.355936,27.710284,NaN,NaN,Apt 361
4,7843 S Euclid Ave,Chicago,IL,60649,"7843 S Euclid Ave, Apt 1, Chicago, IL 60649",6.0,3345.0,1925.0,"{'architectureType': 'Two Family', 'cooling': ...",Cook,...,{'2020': {'total': 2527}},225000.0,2022-09-23T00:00:00.000Z,"{'names': ['78TH EUCLID LLC,'], 'mailingAddres...","7843-S-Euclid-Ave,-Apt-1,-Chicago,-IL-60649",-87.576879,41.752132,SOUTHFIELD,NaN,Apt 1


###   DATA CLEANING AND TRASNFORMATION

In [ ]:
#real_estate_df.info()

In [4]:
# REPLACE MISSING VALUES WITH APPROPRIATE REPLACEMENTS
real_estate_df.fillna({
        'bathrooms': 0.0,
        'bedrooms' : 0.0,
        'squareFootage' :0.0,
        'county': 'uknown',
        'propertyType': 'unknown',
        'yearBuilt' : 0.0,  
        'assessorID' : 'unkmown', 
        'legalDescription' : 'unknown',   
        'subdivision' :'unknown',
        'lotSize' : 0.0, 
        'ownerOccupied' : 0.0,
        'features' : 'unknown',
        'taxAssessment' : 'unknown',
        'propertyTaxes' : 'unknown',  
        'owner'         : 'unknown',
        'zoning'        : 'unknown', 
        'lastSalePrice' : 0.0,
        'addressLine2' : 'unknown'
},inplace=True)

STEP 3 MODELLING OF DATA

CREATING DIMENSION TABLES

In [5]:
real_estate_df.columns

Index(['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'bedrooms', 'squareFootage', 'yearBuilt', 'features', 'county',
       'assessorID', 'legalDescription', 'ownerOccupied', 'bathrooms',
       'lotSize', 'propertyType', 'taxAssessment', 'propertyTaxes',
       'lastSalePrice', 'lastSaleDate', 'owner', 'id', 'longitude', 'latitude',
       'subdivision', 'zoning', 'addressLine2'],
      dtype='object')

In [6]:
location_dim = real_estate_df[['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress','county',
                              'longitude', 'latitude','addressLine2']].copy().drop_duplicates().reset_index(drop=True)
# to add a column that was not in existence use the below code
location_dim.index.name = 'location_id'
# USE THIS CODE TO MAKE THE NEW CREATED INDEX TO APPEAR ON THE COLUMN HEADER
location_dim = location_dim.reset_index()

In [ ]:
#location_dim.head()

In [7]:
sales_dim = real_estate_df[['lastSaleDate','lastSalePrice']].copy().drop_duplicates().reset_index(drop=True)
sales_dim.index.name = 'sales_id'
sales_dim = sales_dim.reset_index()

In [8]:
# the features table has other tables in it that is in form of dictionary
# SOLUTION : this table has to be converted to a string or you expand the dictionary

real_estate_df['features'] = real_estate_df['features'].astype(str)
real_estate_df['taxAssessment'] = real_estate_df['taxAssessment'].astype(str)
real_estate_df['propertyTaxes'] = real_estate_df['propertyTaxes'].astype(str)

In [9]:
features_dim = real_estate_df[['bedrooms','bathrooms','squareFootage','lotSize',
                               'features']].copy().drop_duplicates().reset_index(drop=True)
features_dim.index.name = 'features_id'
features_dim = features_dim.reset_index()


In [10]:
# PROPERTY FACT TABLE
# remember to pick the colums you need from the property table after merging the dimension tables
property_fact_table = real_estate_df.merge(sales_dim, on=['lastSaleDate','lastSalePrice'],how = 'left')\
                                    .merge(location_dim, on=['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress','county','longitude', 'latitude','addressLine2'],how = 'left')\
                                    .merge(features_dim, on=['bedrooms','bathrooms','squareFootage','lotSize','features'],how='left')\
                                    [['id','sales_id','location_id', 'features_id','yearBuilt', 'assessorID','legalDescription', 'ownerOccupied','propertyType','taxAssessment', 'propertyTaxes','subdivision','zoning']]


In [ ]:
property_fact_table.info()

In [ ]:
#real_estate_df.head()

In [ ]:
#features_dim.head()

STEP 4 SAVE ALL TABLES INTO CSV

In [44]:
location_dim.to_csv('Transformed_file/location_dim.csv', index=False)
sales_dim.to_csv('Transformed_file/sales_dim.csv', index=False)
features_dim.to_csv('Transformed_file/features_dim.csv', index=False)
property_fact_table.to_csv('Transformed_file/property_fact_table.csv', index=False)

#### STEP 5 LOADING LAYER
#### OPEN PGADMIN SOFTWARE CLICK ON DATABASE AND CREATE A NEW DATABASE 

In [15]:
# DEVELOPE A FUNCTION TO GET THE DATABASE CONNECTION
# Load environment variables from .env file
# Load environment variables from .env file
load_dotenv()

def get_db_connection():
    connection = psycopg2.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USERNAME'),
        password=os.getenv('DB_PASSWORD'),
        port=os.getenv('DB_PORT')  # Add the port here
    ) 
    return connection

# connect to database
conn = get_db_connection()

In [16]:
# CREATE A FUNCTION THAT SETS UP THE SCHEMA AND TABLE
def create_tables():
    conn = get_db_connection()
    cursor =conn.cursor()
    create_table_query = '''
                            CREATE SCHEMA IF NOT EXISTS RYCHENY_REALTOR;

                            DROP TABLE IF EXISTS RYCHENY_REALTOR.location_dim CASCADE;
                            DROP TABLE IF EXISTS RYCHENY_REALTOR.sales_dim CASCADE;
                            DROP TABLE IF EXISTS RYCHENY_REALTOR.features_dim CASCADE;
                            DROP TABLE IF EXISTS RYCHENY_REALTOR.property_fact_table CASCADE;

                            CREATE TABLE RYCHENY_REALTOR.location_dim(
                            location_id INTEGER PRIMARY KEY,
                            addressLine1 VARCHAR(10000),
                            city VARCHAR(1000),
                            state VARCHAR(1000),
                            zipCode INTEGER,
                            formattedAddress VARCHAR(10000),
                            county VARCHAR(1000),
                            longitude FLOAT,
                            latitude FLOAT,
                            addressLine2 VARCHAR(1000)
                            );

                            CREATE TABLE RYCHENY_REALTOR.sales_dim(
                            sales_id INTEGER PRIMARY KEY,
                            lastSaleDate  VARCHAR(10000),
                            lastSalePrice VARCHAR(1000)
                            );

                            CREATE TABLE RYCHENY_REALTOR.features_dim(
                            features_id INTEGER PRIMARY KEY,
                            bedrooms FLOAT,
                            bathrooms FLOAT,
                            squareFootage FLOAT,
                            lotSize FLOAT,
                            features  VARCHAR(10000)
                            );

                            CREATE TABLE RYCHENY_REALTOR.property_fact_table(
                            id VARCHAR(10000) PRIMARY KEY,
                            sales_id INTEGER,
                            location_id INTEGER,
                            features_id INTEGER,
                            yearBuilt  FLOAT,
                            assessorID VARCHAR(10000),
                            legalDescription VARCHAR(1000),
                            ownerOccupied FLOAT,
                            propertyType VARCHAR(1000),
                            taxAssessment VARCHAR(1000),
                            propertyTaxes VARCHAR(1000),
                            subdivision VARCHAR(1000),
                            zoning VARCHAR(1000),
                            FOREIGN KEY (location_id) REFERENCES RYCHENY_REALTOR.location_dim(location_id),
                            FOREIGN KEY (sales_id) REFERENCES RYCHENY_REALTOR.sales_dim(sales_id),
                            FOREIGN KEY (features_id) REFERENCES RYCHENY_REALTOR.features_dim(features_id)
                            );
                            '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

In [17]:
create_tables()

In [18]:
import logging
import psycopg2  # or the relevant database adapter

# Configure logging to log errors
logging.basicConfig(filename='db_insert_errors.log', level=logging.ERROR)

# Initialize the database connection and cursor
try:
    conn = psycopg2.connect(
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT')
    )
    cursor = conn.cursor()

    # Iterate over the rows in the DataFrame and insert them into the database
    for _, row in property_fact_table.iterrows():
        try:
            # Check if location_id exists in location_dim
            cursor.execute('SELECT 1 FROM RYCHENY_REALTOR.location_dim WHERE id = %s', (row['location_id'],))
            if cursor.fetchone() is None:
                raise ValueError(f"location_id {row['location_id']} does not exist in location_dim")

            # If the check passes, insert the row
            cursor.execute(
                '''INSERT INTO RYCHENY_REALTOR.property_fact_table(id, sales_id, location_id, features_id, yearBuilt,
                assessorID, legalDescription, ownerOccupied, propertyType, taxAssessment, propertyTaxes, subdivision, zoning)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
                (row['id'], row['sales_id'], row['location_id'], row['features_id'], row['yearBuilt'], row['assessorID'],
                 row['legalDescription'], row['ownerOccupied'], row['propertyType'], row['taxAssessment'], row['propertyTaxes'], row['subdivision'], row['zoning'])
            )
        except Exception as e:
            # Log the error and continue with the next row
            logging.error(f"Error inserting row with id {row['id']}: {e}")

    # Commit changes
    conn.commit()

except Exception as e:
    # Log any connection errors
    logging.error(f"Database connection failed: {e}")

finally:
    # Ensure that the cursor and connection are properly closed
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()

In [19]:
# Loading data into db tables
conn = get_db_connection()
cursor = conn.cursor()

# Insert the dataframe into SQL tables using executemany for better performance
cursor.executemany(
    '''INSERT INTO RYCHENY_REALTOR.location_dim(location_id, addressLine1, city, state, zipCode, formattedAddress, county,
                        longitude, latitude, addressLine2)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
    location_dim.values.tolist()
)

cursor.executemany(
    '''INSERT INTO RYCHENY_REALTOR.sales_dim(sales_id, lastSaleDate, lastSalePrice)
        VALUES (%s, %s, %s)''',
    sales_dim[['sales_id', 'lastSaleDate', 'lastSalePrice']].values.tolist()
)

cursor.executemany(
    '''INSERT INTO RYCHENY_REALTOR.features_dim(features_id, bedrooms, bathrooms, squareFootage, lotSize, features)
        VALUES (%s, %s, %s, %s, %s, %s)''',
    features_dim[['features_id', 'bedrooms', 'bathrooms', 'squareFootage', 'lotSize', 'features']].values.tolist()
)

cursor.executemany(
    '''INSERT INTO RYCHENY_REALTOR.property_fact_table(id, sales_id, location_id, features_id, yearBuilt,
    assessorID, legalDescription, ownerOccupied, propertyType, taxAssessment, propertyTaxes, subdivision, zoning)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)''',
    property_fact_table.values.tolist()
)

# Commit changes
conn.commit()

# Close connection
cursor.close()
conn.close()


In [ ]:
location_dim.head()

In [ ]:
#location_dim.info()

 STEP 5 LOADING INTO THE DATABASE ON POSTGRESQL 

In [ ]:
# import pandas as pd

# Assuming property_fact_table is your DataFrame
# Check the data types
#print(property_fact_table.dtypes)

# Inspect the problematic row(s)
#print(property_fact_table[property_fact_table['id'].apply(lambda x: isinstance(x, str))])

# Convert 'id' column to integer, if it should be an integer
#property_fact_table['id'] = pd.to_numeric(property_fact_table['id'], errors='coerce')

# Alternatively, you might want to handle specific cases or rows
# E.g., replace non-numeric ids with a default value or drop those rows
#property_fact_table = property_fact_table.dropna(subset=['id'])  # Drop rows where 'id' could not be converted
#property_fact_table['id'] = property_fact_table['id'].astype(int)

In [ ]:
display(property_fact_table.head())